In [1]:
import numpy as np
import tensorflow as tf

In [2]:
max_seq_len = 2
embedding_dim = 3

- None : 데이터 개수
- max_seq_len : 가장 긴 문장의 길이
- embedding_dim : 노드 개수 (차원 수) -> 실제로는 300 정도 씀

In [3]:
X = tf.placeholder(tf.float32, [None, max_seq_len, embedding_dim])
seq_length = tf.placeholder(tf.int32, [None])

In [4]:
X_batch = np.array([
    [[0,1,2], [9,8,7]], # data 1
    [[3,4,5], [0,0,0]], # data 2
    [[6,7,8], [6,5,4]], # data 3
    [[9,0,1], [3,2,1]], # data 4
])

In [5]:
seq_length_batch = np.array([2,1,2,2]) # 원래 데이터 길이

In [6]:
lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(5, forget_bias=1.0, state_is_tuple=True)
lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(5, forget_bias=1.0, state_is_tuple=True)

In [7]:
(output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, 
                                                                 dtype=tf.float32, inputs=X, sequence_length=seq_length_batch)

In [8]:
init = tf.global_variables_initializer()

In [9]:
with tf.Session() as sess:
    sess.run(init)
    fw, bw, state = sess.run([output_fw, output_bw, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

In [14]:
print(fw)
print('shape of fw : ', np.shape(fw))

[[[-1.16563685e-01 -2.03638375e-01  6.74078416e-04 -2.62705415e-01
    7.57928118e-02]
  [-5.21051228e-01 -5.14112175e-01  2.29265704e-03 -1.79767460e-01
   -6.38808489e-01]]

 [[-4.90682811e-01 -3.58689994e-01  4.49464284e-03 -5.67612410e-01
   -3.30198228e-01]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]]

 [[-6.04616940e-01 -3.78677785e-01  1.31933694e-03 -6.61060154e-01
   -6.32997692e-01]
  [-5.25042295e-01 -5.85256279e-01 -7.81590864e-03 -5.15262306e-01
   -7.31049001e-01]]

 [[-9.49704647e-02 -1.19395526e-02  4.45333868e-02  3.47391188e-01
   -4.36972111e-01]
  [-3.46656740e-01 -3.39600146e-01  1.10614918e-01  6.37855589e-01
   -4.55675960e-01]]]
shape of fw :  (4, 2, 5)


In [16]:
print(bw)
print('shape of bw : ', np.shape(bw))

[[[-3.57179530e-02  7.83808902e-02  1.56682104e-01 -4.57630605e-01
    1.86086521e-02]
  [ 2.24752516e-01  6.98374652e-06  2.27263924e-02 -6.35342360e-01
   -7.89321288e-02]]

 [[-1.36436090e-01  2.96449987e-03  1.41109467e-01 -3.55418921e-01
    5.58703355e-02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]]

 [[-1.32820949e-01  1.19496275e-04  1.32967606e-01 -8.07458401e-01
   -6.04730286e-02]
  [ 2.04131097e-01  5.03828749e-04  5.18747382e-02 -6.01215065e-01
   -1.40585974e-01]]

 [[ 2.89562404e-01  5.14393905e-03  2.31614597e-02 -8.77732635e-01
   -4.87606317e-01]
  [ 1.65443748e-01  1.68687142e-02  1.00280710e-01 -4.60649371e-01
   -1.72178045e-01]]]
shape of bw :  (4, 2, 5)


c = cell state
h = hidden state

In [26]:
print(state)
print('shape of state : ', np.shape(state))

(LSTMStateTuple(c=array([[-1.166706  , -1.4321389 ,  0.00230553, -0.1822118 , -0.85896903],
       [-0.80001163, -0.9724843 ,  0.00464034, -0.6674181 , -0.42097268],
       [-1.8683853 , -1.7783544 , -0.00820133, -0.5848301 , -1.7053621 ],
       [-1.1307411 , -0.59057915,  0.16678306,  1.2495613 , -1.3759655 ]],
      dtype=float32), h=array([[-0.5210512 , -0.5141122 ,  0.00229266, -0.17976746, -0.6388085 ],
       [-0.4906828 , -0.35869   ,  0.00449464, -0.5676124 , -0.33019823],
       [-0.5250423 , -0.5852563 , -0.00781591, -0.5152623 , -0.731049  ],
       [-0.34665674, -0.33960015,  0.11061492,  0.6378556 , -0.45567596]],
      dtype=float32)), LSTMStateTuple(c=array([[-0.10666889,  0.2710605 ,  0.27665752, -0.8810166 ,  0.02394943],
       [-0.38868818,  0.03198289,  0.41364285, -0.40326905,  0.05740749],
       [-0.32168874,  0.00667208,  1.0307295 , -1.1376646 , -0.06067512],
       [ 0.30587912,  0.03867058,  0.8877984 , -1.4940069 , -0.9598786 ]],
      dtype=float32), h=arr

함수 사용 법

In [ ]:
tf.nn.bidirectional_dynamic_rnn?

hidden state 만 모아서 합쳐준다.

In [27]:
final_state = tf.concat([state[0][1], state[1][1]], axis=1)

In [29]:
with tf.Session() as sess:
    print(sess.run(final_state))

[[-5.21051228e-01 -5.14112175e-01  2.29265704e-03 -1.79767460e-01
  -6.38808489e-01 -3.57179530e-02  7.83808902e-02  1.56682104e-01
  -4.57630605e-01  1.86086521e-02]
 [-4.90682811e-01 -3.58689994e-01  4.49464284e-03 -5.67612410e-01
  -3.30198228e-01 -1.36436090e-01  2.96449987e-03  1.41109467e-01
  -3.55418921e-01  5.58703355e-02]
 [-5.25042295e-01 -5.85256279e-01 -7.81590864e-03 -5.15262306e-01
  -7.31049001e-01 -1.32820949e-01  1.19496275e-04  1.32967606e-01
  -8.07458401e-01 -6.04730286e-02]
 [-3.46656740e-01 -3.39600146e-01  1.10614918e-01  6.37855589e-01
  -4.55675960e-01  2.89562404e-01  5.14393905e-03  2.31614597e-02
  -8.77732635e-01 -4.87606317e-01]]
